In [2]:
DATA_PREPARATION_VOTE_METHOD = "max_vote_window" # "max_vote_window" or "sum_and_normalize". Decides how to aggregate the predictions of the overlapping windows
PRETRAINED_MODEL_NAME_OR_PATH = "tf_efficientnet_b0_ns"

In [ ]:
import sys
import os

from tqdm import tqdm 
import wandb
import torch

import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.amp import GradScaler, autocast
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR


if bool(os.environ.get("KAGGLE_URL_BASE", "")):
  import sys
  # running on kaggle
  sys.path.insert(0, "/kaggle/input/hsm-source-files")
else:
  # running locally
  sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..")))

from src.datasets.multi_spectrogram import MultiSpectrogramDataset
from src.utils.k_folds_creator import KFoldCreator
from src.utils.utils import get_models_save_path, set_seeds, get_raw_data_dir, get_processed_data_dir
from src.models.base_cnn import BaseCNN
from src.utils.constants import Constants 
from src.datasets.eeg_processor import EEGDataProcessor

In [ ]:
wandb.login()

In [5]:
class CFG:
    seed = 42
    n_splits = 5
    data_path = get_raw_data_dir()
    train_eeg_spec_path = get_processed_data_dir() / "eeg_spectrograms" / "train" / "cwt"
    
    model_name = PRETRAINED_MODEL_NAME_OR_PATH
    in_channels = 4  
    target_size = 6 
    
    batch_size = 32
    num_workers = 8
    epochs = 5
    lr = 1e-3
    
    img_size = (128, 256)

set_seeds(42)

In [ ]:
TARGETS = Constants.TARGETS

processor = EEGDataProcessor(raw_data_path=CFG.data_path, processed_data_path=get_processed_data_dir())

In [7]:
def get_dataloaders(df, fold_id):
    train_df = df[df['fold'] != fold_id].reset_index(drop=True)
    valid_df = df[df['fold'] == fold_id].reset_index(drop=True)

    train_dataset = MultiSpectrogramDataset(
        train_df, TARGETS, CFG.data_path, CFG.img_size, CFG.train_eeg_spec_path, mode='train'
    )
    valid_dataset = MultiSpectrogramDataset(
        valid_df, TARGETS, CFG.data_path, CFG.img_size, CFG.train_eeg_spec_path, mode='train'
    )
    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG.batch_size,
        shuffle=True,
        num_workers=CFG.num_workers,
        pin_memory=True,
        drop_last=True,
        persistent_workers=True
    )
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=CFG.batch_size,
        shuffle=False,
        num_workers=CFG.num_workers,
        pin_memory=True,
        drop_last=False,
        persistent_workers=True
    )
    
    return train_loader, valid_loader

In [8]:
def run_training(df, data_preparation_vote_method):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    scaler = GradScaler(enabled=(device.type == "cuda"))

    all_oof_preds = []
    all_oof_labels = []
    for fold in range(CFG.n_splits):
        print(f"\n========== FOLD {fold} ==========")

        config = {
            # Model
            "architecture": CFG.model_name,
            "pretrained": True,
            # Data
            "fold": fold,
            "features": "multi_spectrograms",
            "window_selection": DATA_PREPARATION_VOTE_METHOD,
            # Training
            "optimizer": "AdamW",
            "learning_rate": CFG.lr,
            "batch_size": CFG.batch_size,
            "epochs": CFG.epochs,
            "seed": CFG.seed,
            "scheduler": "CosineAnnealingLR" 
        }

        wandb.init(
            project="hms-aicomp-cnn-multispec",
            name=f"{CFG.model_name}-multispec-fold{fold}", 
            tags=[f'fold{fold}'],
            config=config
        )

        model = BaseCNN(CFG.model_name, pretrained=True, num_classes=CFG.target_size)
        model.to(device)

        optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.lr)
        scheduler = CosineAnnealingLR(optimizer, T_max=CFG.epochs)
        loss_fn = nn.KLDivLoss(reduction='batchmean')
        train_loader, valid_loader = get_dataloaders(df, fold)

        best_val_loss = float('inf')
        best_model_path = None

        for epoch in range(CFG.epochs):
            print(f"  --- Epoch {epoch+1}/{CFG.epochs} ---")

            model.train()
            train_loss = 0
            for images, labels in tqdm(train_loader, desc="Training"):
                images, labels = images.to(device), labels.to(device)

                optimizer.zero_grad()
                with autocast(device_type=device.type, dtype=torch.float16):
                    outputs = model(images)
                    log_probs = F.log_softmax(outputs, dim=1)
                    loss = loss_fn(log_probs, labels)

                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()

                train_loss += loss.item() * images.size(0)

                wandb.log({"train/loss": loss.item()})

            train_loss /= len(train_loader.dataset)

            model.eval()
            valid_loss = 0
            with torch.no_grad():
                for images, labels in tqdm(valid_loader, desc="Validation"):
                    images, labels = images.to(device), labels.to(device)

                    with autocast(device_type=device.type, dtype=torch.float16):
                        outputs = model(images)
                        log_probs = F.log_softmax(outputs, dim=1)
                        loss = loss_fn(log_probs, labels)

                    valid_loss += loss.item() * images.size(0)

            valid_loss /= len(valid_loader.dataset)
            
            epoch_lr = optimizer.param_groups[0]['lr']
            print(f"   Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Valid Loss = {valid_loss:.4f}, LR = {epoch_lr:.6f}")

            wandb.log({
                "epoch": epoch + 1,
                "train/epoch_loss": train_loss,
                "val/loss": valid_loss,
                "val/kl_div": valid_loss,
                "train/epoch_lr": epoch_lr
            })

            if valid_loss < best_val_loss:
                best_val_loss = valid_loss
                best_model_path = get_models_save_path() / "multi_spec_cnn" / CFG.model_name / data_preparation_vote_method / f'best_model_fold{fold}.pth'
                best_model_path.parent.mkdir(parents=True, exist_ok=True)
                torch.save(model.state_dict(), best_model_path)
                print(f"  New best model saved with validation loss: {best_val_loss:.4f}")
            scheduler.step()

        print(f"   --- Generating OOF predictions for fold {fold} ---")
        if best_model_path:
            model.load_state_dict(torch.load(best_model_path))
            model.eval()

            fold_oof_preds = []
            fold_oof_labels = []

            with torch.no_grad():
                for images, labels in tqdm(valid_loader, desc=f"OOF Prediction Fold {fold}"):
                    images = images.to(device)
                    outputs = model(images)
                    probs = F.softmax(outputs, dim=1).cpu()

                    fold_oof_preds.append(probs)
                    fold_oof_labels.append(labels.cpu())

            all_oof_preds.append(torch.cat(fold_oof_preds).numpy())
            all_oof_labels.append(torch.cat(fold_oof_labels).numpy())
            print(f"   Finished OOF predictions for fold {fold}")
        else:
            raise RuntimeError("Best model path is None, cannot generate OOF predictions.")


        wandb.summary['best_val_kl_div'] = best_val_loss

        if best_model_path:
            artifact = wandb.Artifact(f'{CFG.model_name}-fold{fold}', type='model')
            artifact.add_file(best_model_path)
            wandb.log_artifact(artifact)
            print(f"\nLogged artifact for fold {fold} with best validation loss: {best_val_loss:.4f}")
        else:
            print("\nNo best model was saved during training for this fold.")

        wandb.finish()

    if all_oof_preds and all_oof_labels:
        print("\nCalculating final OOF score...")
        final_oof_preds = np.concatenate(all_oof_preds)
        final_oof_labels = np.concatenate(all_oof_labels)

        oof_preds_tensor = torch.tensor(final_oof_preds, dtype=torch.float32)
        oof_labels_tensor = torch.tensor(final_oof_labels, dtype=torch.float32)

        log_oof_preds_tensor = torch.log(oof_preds_tensor)

        kl_loss_fn = nn.KLDivLoss(reduction='batchmean')
        overall_oof_score = kl_loss_fn(log_oof_preds_tensor, oof_labels_tensor).item()

        print(f"\nOverall OOF KL Score: {overall_oof_score:.4f}")
    else:
        print("\nCould not calculate OOF score because no predictions were generated.")
        
    return overall_oof_score

## Run Training

In [ ]:
print("Preparing data and creating folds...")
train_df = processor.process_data(vote_method=DATA_PREPARATION_VOTE_METHOD, skip_parquet=True)
print('Train shape:', train_df.shape)
print('Targets', list(TARGETS))

fold_creator = KFoldCreator(n_splits=CFG.n_splits, seed=CFG.seed)
train_df = fold_creator.create_folds(train_df, stratify_col='expert_consensus', group_col='patient_id')

print("Folds created. Value counts per fold:")
print(train_df['fold'].value_counts())

In [ ]:
overall_oof_score = run_training(train_df, DATA_PREPARATION_VOTE_METHOD)

wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.


In [ ]:
print(f"Overall OOF KL Score from training: {overall_oof_score:.4f}")